# Augmentation

This notebook is used to augment the dataset with new images. The augmentation is done using the `albumentations` library, which provides a wide range of image transformations.

In [ ]:
import os
import cv2
import albumentations as A

In [ ]:
def load_labels(label_path):
    bboxes = []
    class_labels = []
    with open(label_path, 'r') as f:
        for line in f.readlines():
            parts = line.strip().split()
            class_id = int(parts[0])
            bbox = list(map(float, parts[1:]))
            bboxes.append(bbox)
            class_labels.append(class_id)
    return bboxes, class_labels

In [ ]:
def save_augmented_data(image, bboxes, class_labels, out_img_path, out_label_path):
    cv2.imwrite(out_img_path, image)
    with open(out_label_path, 'w') as f:
        for cls, bbox in zip(class_labels, bboxes):
            f.write(f"{cls} {' '.join(map(str, bbox))}\n")

In [ ]:
import os
import cv2
import albumentations as A

def load_labels(label_path):
    try:
        bboxes = []
        class_labels = []
        with open(label_path, 'r') as f:
            for line in f.readlines():
                parts = line.strip().split()
                if len(parts) != 5:
                    continue
                class_labels.append(int(parts[0]))
                bboxes.append(list(map(float, parts[1:])))
        return bboxes, class_labels
    except Exception as e:
        print(f"Error loading labels: {label_path}, {e}")
        return [], []

def save_augmented_data(image, bboxes, class_labels, image_path, label_path):
    cv2.imwrite(image_path, image)
    with open(label_path, 'w') as f:
        for cls, box in zip(class_labels, bboxes):
            f.write(f"{cls} {' '.join(map(str, box))}\n")

transformations = [
    A.Compose([A.Rotate(limit=(0, 0), p=1)], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels'])),
    A.Compose([A.Rotate(limit=(90, 90), p=1)], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels'])),
    A.Compose([A.Rotate(limit=(180, 180), p=1)], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels'])),
    A.Compose([A.Rotate(limit=(270, 270), p=1)], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels'])),
]

flip = A.Compose([
    A.HorizontalFlip(p=1)
], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels']))

images_dir = '../preprocessedData/LearnSet/data2/images/train'
labels_dir =  '../preprocessedData/LearnSet/data2/labels/train'
output_images_dir = '../preprocessedData/LearnSet/dataAugumented/images/train'
output_labels_dir = '../preprocessedData/LearnSet/dataAugumented/labels/train'

os.makedirs(images_dir, exist_ok=True)
os.makedirs(labels_dir, exist_ok=True)
os.makedirs(output_images_dir, exist_ok=True)
os.makedirs(output_labels_dir, exist_ok=True)

for filename in os.listdir(images_dir):
    if not filename.lower().endswith(('.jpg', '.png', '.jpeg', '.tif', '.tiff')):
        continue

    img_path = os.path.join(images_dir, filename)
    label_path = os.path.join(labels_dir, filename.rsplit('.', 1)[0] + '.txt')

    image = cv2.imread(img_path)
    if image is None:
        print(f"Cannot read image: {img_path}")
        continue

    bboxes, class_labels = load_labels(label_path)
    base_name = filename.rsplit('.', 1)[0]

    success_count = 0

    for i, rot in enumerate(transformations):
        try:
            result = rot(image=image, bboxes=bboxes, class_labels=class_labels)

            out_name = f"{base_name}_rot{i}"
            save_augmented_data(
                result['image'],
                result['bboxes'],
                result['class_labels'],
                os.path.join(output_images_dir, out_name + '.tif'),
                os.path.join(output_labels_dir, out_name + '.txt')
            )
            success_count += 1

            try:
                flipped = flip(image=result['image'], bboxes=result['bboxes'], class_labels=result['class_labels'])
                out_name_flip = f"{base_name}_rot{i}_flip"
                save_augmented_data(
                    flipped['image'],
                    flipped['bboxes'],
                    flipped['class_labels'],
                    os.path.join(output_images_dir, out_name_flip + '.tif'),
                    os.path.join(output_labels_dir, out_name_flip + '.txt')
                )
                success_count += 1
            except Exception as e:
                print(f"Flip failed for {filename} rot{i}: {e}")

        except Exception as e:
            print(f"Rotation {i} failed for {filename}: {e}")

    if success_count == 0:
        print(f"No augmentations succeeded for {filename}, saving original.")
        out_img = os.path.join(output_images_dir, base_name + '_original.tif')
        out_lbl = os.path.join(output_labels_dir, base_name + '_original.txt')
        save_augmented_data(image, bboxes, class_labels, out_img, out_lbl)
        success_count = 1

    print(f"{filename}: {success_count} augmentations succeeded.")


In [ ]:
images_dir = '../preprocessedData/LearnSet/data/images/train'
labels_dir = '../preprocessedData/LearnSet/data/labels/train'
output_dir = '../preprocessedData/LearnSet/vis_checked'  


In [ ]:
import os
import cv2
import albumentations as A

def load_polygons(label_path):
    polygons = []
    try:
        with open(label_path, 'r') as f:
            for line in f:
                line = line.strip()
                if not line:
                    continue
                points_str = line.split(';')
                points = []
                for p in points_str:
                    if ',' not in p:
                        continue
                    x_str, y_str = p.split(',')
                    points.append((float(x_str), float(y_str)))
                if points:
                    polygons.append(points)
        return polygons
    except Exception as e:
        print(f"Error loading polygons: {label_path}, {e}")
        return []

def save_polygons(polygons, label_path):
    with open(label_path, 'w') as f:
        for poly in polygons:
            line = ';'.join([f"{x:.2f},{y:.2f}" for x, y in poly])
            f.write(line + '\n')

transformations = [
    A.Compose([A.Rotate(limit=(0, 0), p=1)], 
              keypoint_params=A.KeypointParams(format='xy', remove_invisible=False)),
    A.Compose([A.Rotate(limit=(90, 90), p=1)], 
              keypoint_params=A.KeypointParams(format='xy', remove_invisible=False)),
    A.Compose([A.Rotate(limit=(180, 180), p=1)], 
              keypoint_params=A.KeypointParams(format='xy', remove_invisible=False)),
    A.Compose([A.Rotate(limit=(270, 270), p=1)], 
              keypoint_params=A.KeypointParams(format='xy', remove_invisible=False)),
]

flip = A.Compose([
    A.HorizontalFlip(p=1)
], keypoint_params=A.KeypointParams(format='xy', remove_invisible=False))

def polygons_to_keypoints(polygons):

    keypoints = []
    polygon_indexes = []
    for poly_idx, poly in enumerate(polygons):
        for point in poly:
            keypoints.append(point)
            polygon_indexes.append(poly_idx)
    return keypoints, polygon_indexes

def keypoints_to_polygons(keypoints, polygon_indexes):
    polygons_dict = {}
    for pt, idx in zip(keypoints, polygon_indexes):
        polygons_dict.setdefault(idx, []).append(pt)
    polygons_out = [polygons_dict[i] for i in sorted(polygons_dict.keys())]
    return polygons_out

images_dir = 'C:/Users/stszy/PycharmProjects/glandsDetector/preprocessedData_v1/LearnSet/data/images/train'
labels_dir =  'C:/Users/stszy/PycharmProjects/glandsDetector/preprocessedData_v1/LearnSet/data/labels/train'
output_images_dir = 'C:/Users/stszy/PycharmProjects/glandsDetector/preprocessedData_v1/LearnSet/data/val/images/train'
output_labels_dir = 'C:/Users/stszy/PycharmProjects/glandsDetector/preprocessedData_v1/LearnSet/data/val/labels/train'

os.makedirs(output_images_dir, exist_ok=True)
os.makedirs(output_labels_dir, exist_ok=True)

for filename in os.listdir(images_dir):
    if not filename.lower().endswith(('.jpg', '.png', '.jpeg', '.tif', '.tiff')):
        continue

    img_path = os.path.join(images_dir, filename)
    label_path = os.path.join(labels_dir, filename.rsplit('.', 1)[0] + '.txt')

    image = cv2.imread(img_path)
    if image is None:
        print(f"Cannot read image: {img_path}")
        continue

    polygons = load_polygons(label_path)
    if not polygons:
        print(f"No polygons found for {filename}")
        continue

    base_name = filename.rsplit('.', 1)[0]
    success_count = 0

    keypoints, polygon_indexes = polygons_to_keypoints(polygons)

    for i, transform in enumerate(transformations):
        try:
            augmented = transform(image=image, keypoints=keypoints)
            new_image = augmented['image']
            new_keypoints = augmented['keypoints']

            new_polygons = keypoints_to_polygons(new_keypoints, polygon_indexes)

            out_name = f"{base_name}_rot{i}"
            cv2.imwrite(os.path.join(output_images_dir, out_name + '.tif'), new_image)
            save_polygons(new_polygons, os.path.join(output_labels_dir, out_name + '.txt'))
            success_count += 1

            try:
                flipped = flip(image=new_image, keypoints=new_keypoints)
                flipped_image = flipped['image']
                flipped_keypoints = flipped['keypoints']
                flipped_polygons = keypoints_to_polygons(flipped_keypoints, polygon_indexes)

                out_name_flip = f"{base_name}_rot{i}_flip"
                cv2.imwrite(os.path.join(output_images_dir, out_name_flip + '.tif'), flipped_image)
                save_polygons(flipped_polygons, os.path.join(output_labels_dir, out_name_flip + '.txt'))
                success_count += 1
            except Exception as e:
                print(f"Flip failed for {filename} rot{i}: {e}")

        except Exception as e:
            print(f"Rotation {i} failed for {filename}: {e}")

    if success_count == 0:
        print(f"No augmentations succeeded for {filename}, saving original.")
        out_img = os.path.join(output_images_dir, base_name + '_original.tif')
        out_lbl = os.path.join(output_labels_dir, base_name + '_original.txt')
        cv2.imwrite(out_img, image)
        save_polygons(polygons, out_lbl)
        success_count = 1

    print(f"{filename}: {success_count} augmentations succeeded.")


Function to check if any bounding box has negative values in the labels directory.


In [ ]:
import os

def check_negative_values(labels_dir):
    for filename in os.listdir(labels_dir):
        if not filename.endswith('.txt'):
            continue

        label_path = os.path.join(labels_dir, filename)
        bboxes, class_labels = load_labels(label_path)

        for bbox in bboxes:
            
            if any(coord < 0 for coord in bbox[:4]): 
                print(f"Negative value found in file: {label_path}")
                print(f"Bounding box: {bbox}")

In [ ]:
check_negative_values(labels_dir)


# Function to draw bounding boxes snd polygons on images to check correctness

In [6]:
import os
import cv2

def draw_boxes_for_folder(images_dir, labels_dir, output_dir=None, class_names=None, show=False):
    os.makedirs(output_dir, exist_ok=True) if output_dir else None

    for filename in os.listdir(images_dir):
        if not filename.lower().endswith(('.jpg', '.png', '.jpeg', '.tif', '.tiff')):
            continue

        image_path = os.path.join(images_dir, filename)
        label_name = os.path.splitext(filename)[0] + ".txt"
        label_path = os.path.join(labels_dir, label_name)

        image = cv2.imread(image_path)
        if image is None:
            print(f"Loading failed: {image_path}")
            continue

        height, width = image.shape[:2]

        if not os.path.exists(label_path):
            print(f"No boxes for: {filename}")
            continue

        with open(label_path, 'r') as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) != 5:
                    continue
                class_id, x_center, y_center, w, h = map(float, parts)
                
                x_center *= width
                y_center *= height
                w *= width
                h *= height
                x1 = int(x_center - w / 2)
                y1 = int(y_center - h / 2)
                x2 = int(x_center + w / 2)
                y2 = int(y_center + h / 2)

                color = (0, 255, 0)
                label = class_names[int(class_id)] if class_names else str(int(class_id))

                cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)
                cv2.putText(image, label, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)

        if show:
            cv2.imshow('Image', image)
            cv2.waitKey(0)
        if output_dir:
            output_path = os.path.join(output_dir, filename)
            cv2.imwrite(output_path, image)

    if show:
        cv2.destroyAllWindows()


In [1]:
import os
import cv2
import numpy as np

def draw_contours_for_folder(images_dir, labels_dir, output_dir=None, show=False, color=(0, 255, 0)):
    if output_dir:
        os.makedirs(output_dir, exist_ok=True)

    for filename in os.listdir(images_dir):
        if not filename.lower().endswith(('.jpg', '.png', '.jpeg', '.tif', '.tiff')):
            continue

        image_path = os.path.join(images_dir, filename)
        label_name = os.path.splitext(filename)[0] + ".txt"
        label_path = os.path.join(labels_dir, label_name)

        image = cv2.imread(image_path)
        if image is None:
            print(f"Load failed : {image_path}")
            continue

        if not os.path.exists(label_path):
            print(f"There is no file as {filename}")
            continue

        with open(label_path, 'r') as f:
            for line in f:
                line = line.strip()
                if not line:
                    continue
                
                try:
                    points = [
                        [float(x), float(y)]
                        for x, y in (pair.split(',') for pair in line.split(';'))
                    ]
                except ValueError:
                    print(f"Error in data {label_path}")
                    continue

                contour = np.array(points, dtype=np.int32).reshape((-1, 1, 2))

                cv2.polylines(image, [contour], isClosed=True, color=color, thickness=2)

        if show:
            cv2.imshow("Contour", image)
            cv2.waitKey(0)

        if output_dir:
            output_path = os.path.join(output_dir, filename)
            cv2.imwrite(output_path, image)

    if show:
        cv2.destroyAllWindows()


In [2]:
images_dir = '../preprocessedData/LearnSet/dataAugumented/images/train' #dobre dane
labels_dir = '../preprocessedData/LearnSet/dataAugumented/labels/train'
output_dir = '../preprocessedData/LearnSet/vis_checked'

In [4]:
draw_contours_for_folder(images_dir, labels_dir, output_dir=output_dir, show=False)

KeyboardInterrupt: 

In [7]:
draw_boxes_for_folder(images_dir, labels_dir, output_dir=output_dir+"box", show=False,)

KeyboardInterrupt: 